### FastText文本分类

### train_supervised方法参数

    input             # 训练文件路径(必选参数)(带标签__label__XX的utf-8编码的文本txt)
    lr                # 学习率 [0.1]
    dim               # 词向量维度 [100]
    ws                # 上下文窗口大小 [5]
    epoch             # 训练轮数 [5]
    minCount           # 词出现的最小次数[1]
    minCountLabel        # 标签出现的最小次数 [1]
    minn              # char ngram的最小长度 [0]
    maxn              # char ngram的最大长度 [0]
    neg               # 负采样的数量 [5]
    wordNgrams          # word ngram的最大长度 [1]
    loss              # 损失函数 {ns, hs, softmax, ova} [softmax]
    bucket            # buckets数量 [2000000]
    thread            # threads数量 [number of cpus]
    lrUpdateRate        # 学习率更新的速率 [100]
    t                # 采样阈值 [0.0001]
    label             # 标签前缀形式 ['__label__']
    verbose           # verbose [2]
    pretrainedVectors     # 预训练的词向量 (.vec file) for supervised learning []

In [1]:
import fasttext

In [2]:
train_data = 'data\\train_data.txt'
test_data = 'data\\test_data.txt'

### 训练

In [3]:
model = fasttext.train_supervised(train_data) 

In [4]:
# 属性words 和 labels 返回词典中的词和标签
# model.words         # 相当于 model.get_words() 
model.labels        # 相当于 model.get_labels() # 5类标签：科技，娱乐，体育，军事，汽车

['__label__technology',
 '__label__entertainment',
 '__label__sports',
 '__label__military',
 '__label__car']

In [5]:
# 获取词向量表示
model['国家'] # # equivalent to model.get_word_vector('king')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

### 测试

In [6]:
# 评估模型
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p)) # 精确率
    print("R@{}\t{:.3f}".format(1, r)) # 召回率

print_results(*model.test(test_data))

N	26260
P@1	0.900
R@1	0.900


### 预测 

In [7]:
# 预测特定文本的标签
model.predict(" 2013 年底 阿里 集团 收购 天天 动听 旗下 虾米 音乐 合并 海洋 音乐")

(('__label__entertainment',), array([0.98527485]))

In [8]:
# 默认情况下，predict仅返回一个标签：概率最高的标签。可以通过指定参数来预测多个标签k
model.predict(" 2013 年底 阿里 集团 收购 天天 动听 旗下 虾米 音乐 合并 海洋 音乐", k=3)

(('__label__entertainment', '__label__technology', '__label__sports'),
 array([9.85274851e-01, 1.40851224e-02, 3.73448565e-04]))

In [9]:
# 要预测多个句子，可以传递一个字符串数组
model.predict([" 2013 年底 阿里 集团 收购 天天 动听 旗下 虾米 音乐 合并 海洋 音乐", \
               "本次 赛事 门票 南宁 市民 免费 提供"], k=3)

([['__label__entertainment', '__label__technology', '__label__sports'],
  ['__label__technology', '__label__entertainment', '__label__car']],
 array([[9.85274851e-01, 1.40851224e-02, 3.73448565e-04],
        [8.94260108e-01, 7.15281963e-02, 2.90484205e-02]]))

###  保存模型

In [10]:
model.save_model("model\\news_classifier.bin")

### 压缩模型
quantize(input=None, qout=False, cutoff=0, retrain=False, epoch=None, lr=None, thread=None, verbose=None, dsub=2, qnorm=False)

In [11]:
model.quantize(input=train_data, qnorm=True, retrain=True, cutoff=100000) # 压缩模型
print_results(*model.test(test_data))
model.save_model("model\\news_classifier.ftz")

N	26260
P@1	0.895
R@1	0.895


###  加载模型 

In [12]:
help(fasttext.load_model)

Help on function load_model in module fasttext.FastText:

load_model(path)
    Load a model given a filepath and return a model object.



In [13]:
# 加载模型并预测
model_path = "model\\news_classifier.bin"
model_loaded = fasttext.load_model(model_path) 
model_loaded.predict("CBA 四川 品胜 男篮 主场 广东 宏远 比赛 四川队 大胜 排名 紧紧 咬住 山东 北京 保留 很大 晋级 季后赛")

(('__label__sports',), array([1.00001001]))

### 模型方法

    get_dimension          # 获取lookup vector的维度 (hidden layer).# 等价于 `dim` 属性
    get_input_vector        # 获取给定标签在输入矩阵中相应的向量
    get_input_matrix        # 获取模型的输入矩阵的副本
    get_labels            # 获取字典中的标签列表 # 等价于`labels` 属性
    get_line             # 将一行文本分割为词和标签
    get_output_matrix       # 获取模型的输出矩阵的副本
    get_sentence_vector      # 获取给定字符串的一个向量表示
    get_subword_id         # 获取subword的被哈希到的索引
    get_subwords          # 获取给定词的subwords和其索引
    get_word_id           # 获取给定词在字典中的id
    get_word_vector        # 获取词向量表示
    get_words            # 获取字典中所有词 # 等价于`words`属性
    is_quantized          # 模型是否被压缩
    predict             # 输入待预测字符串，输出一个预测标签和概率的列表
    quantize             # 压缩模型
    save_model            # 保存模型
    test                # 在测试集上评估模型性能
    test_label            # 返回每一类的准确率和召回率 

In [19]:
model_loaded.get_labels(include_freq=True)

(['__label__technology',
  '__label__entertainment',
  '__label__sports',
  '__label__military',
  '__label__car'],
 array([13993, 13984, 13912, 11740,  7671]))

In [22]:
help(model_loaded.get_labels)

Help on method get_labels in module fasttext.FastText:

get_labels(include_freq=False, on_unicode_error='strict') method of fasttext.FastText._FastText instance
    Get the entire list of labels of the dictionary optionally
    including the frequency of the individual labels. Unsupervised
    models use words as labels, which is why get_labels
    will call and return get_words for this type of
    model.

